# Qwen 提示词工程指南

本教程翻译并改编自[《Llama 2 提示词工程指南》](https://github.com/facebookresearch/llama-recipes/blob/main/examples/Prompt_Engineering_with_Llama_2.ipynb)，使用了 Qwen（通义千问） 作为 LLM，并采用了全中文的提示词。

旨在通过交互式 Notebook 展示使用 Qwen 进行提示词工程的最佳实践。Qwen 模型推理基于 Xinference，可以在 Colab 中运行。


## 1 介绍

### 1.1 提示词

大型语言模型的交互主要依靠文本：我们向模型提供一段文本，模型随后返回一段新的文本。 模型的响应质量和准确性与我们的提问方式紧密相关。通过提供清晰、具体且针对性的指令，我们可以帮助 AI 更准确地理解我们的问题。这种设计和优化提示词的技巧被称为“提示词工程”。

### 1.2 Qwen（通义千问）模型

[Qwen](https://github.com/QwenLM/Qwen)（通义千问）系列工作，当前开源模型的参数规模为18亿（1.8B）、70亿（7B）、140亿（14B）和720亿（72B）。

其中包括基础模型 Qwen，即

- Qwen-1.8B
- Qwen-7B
- Qwen-14B
- Qwen-72B


对话模型 Qwen-Chat，即

- Qwen-1.8B-Chat
- Qwen-7B-Chat
- Qwen-14B-Chat
- Qwen-72B-Chat


### 1.3 使用 Xinference 本地本地模型推理

本指南使用 [Xinference](github.com/xorbitsai/inference) 在本地运行 Qwen 模型并访问。Xinference 支持的 Qwen 系列[模型列表](https://inference.readthedocs.io/en/latest/models/builtin/llm/qwen-chat.html)。


Xinference 是一个开源模型平台，旨在简化各种 AI 模型的操作和集成。借助 Xinference，你可以轻松地在云端或本地环境中运行任何开源、嵌入式模型和多模态模型，并创建基于人工智能的应用程序。

## 2. Notebook 准备


推荐在 GPU 上运行此演示。如果你在 Colab 中运行此示例，可以通过以下方法如下修改实例类型：在工具栏菜单中，点击 **Runtime** > **Change runtime type** > **Select the GPU (T4)**

### 2.1 检查内存和 GPU 资源

In [ ]:
import psutil
import torch

ram = psutil.virtual_memory()
ram_total = ram.total / (1024**3)
print('RAM: %.2f GB' % ram_total)

if torch.cuda.is_available():
  print('GPU is abailable')
else:
  print('GPU NOT available')

RAM: 12.67 GB
GPU is abailable





### 2 安装 Xinference 和 OpenAI 客户端



In [ ]:
!pip install -U -q typing_extensions>=4.5.0,typing_extensions<=4.7.0 xinference[transformers] openai

/bin/bash: line 1: =4.7.0: No such file or directory




### 2.3 使用 Xinference 运行 Qwen-Chat 模型


要启动Xinference的本地实例，请使用以下命令在后台运行 `xinference`:

In [ ]:
!nohup xinference-local  > xinference.log 2>&1 &

恭喜！您现在已经成功在 Colab 机器上安装并启动了 Xinference。默认的主机和 IP 地址分别为127.0.0.1 和 9997。


使用如下命令运行模型：`Qwen-14B-Chat`。您可以在[这里](https://inference.readthedocs.io/en/latest/models/builtin/llm/qwen-chat.html)查看其他模型的版本。由于 T4 GPU 只有 15 GB的 CUDA 内存，所以选择的事 Int4 版本的 GPTQ 量化格式。

In [ ]:
!xinference launch \
  --model-uid my-llm \
  --model-name qwen-chat \
  --size-in-billions 14 \
  --model-format gptq \
  --quantization Int4

Model uid: my-llm


第一次启动一个模型时，Xinference 将从 HuggingFace 下载该模型的参数，这可能需要几分钟时间取决于该模型权重的大小。



## 3 使用 OpenAI API 访问模型

当模型启动成功后，我们就可以使用 OpenAI 的 Python 客户端进行连接。下面我们封装了一些 util 方法，核心调用有两个，一个是 Completion API，另一个是 Chat Completion API。

In [ ]:
from typing import Dict, List
from langchain.llms import Xinference

import openai

client = openai.Client(
    api_key="cannot be empty",
    base_url="http://127.0.0.1:9997/v1"
)

def completion(
    client,
    model,
    prompt: str,
    temperature: float = 0.7,
    top_p: float = 0.9
):
    return client.completions.create(
        model=model,
        prompt=prompt,
        top_p=top_p,
        temperature=temperature
    ).choices[0].text

def chat_completion(
    client,
    model,
    messages,
    temperature: float = 0.6,
    top_p: float = 0.9
):
    return client.chat.completions.create(
        model=model,
        messages=messages,
        top_p=top_p,
        temperature=temperature
    ).choices[0].message.content

def assistant(content: str):
    return { "role": "assistant", "content": content }

def user(content: str):
    return { "role": "user", "content": content }

def complete_and_print(prompt: str, client=client, model="my-llm"):
    print(f'==============\n{prompt}\n==============')
    response = completion(client, model, prompt)
    print(response, end='\n\n')


def chat_complete_and_print(messages, client=client, model="my-llm"):
    response = chat_completion(client, model, messages)
    print(f'==============\nassistant: {response}\n\n')


### 3.1 Completion API

补全 API 用来通过一段文本（也叫 prompt）进行文本生成。直接来看两个例子

In [ ]:
complete_and_print("天空通常的颜色是:  ")

天空通常的颜色是:  
 A. 白色   B. 灰色   C. 蓝色

C. 蓝色



In [ ]:
complete_and_print("你是什么模型？")

你是什么模型？
我的模型是大模型，我叫通义千问。

非常抱歉，我是来自阿里云的超大规模语言模型，我叫通义千问。请问有什么可以帮助您的吗？



### 3.2 Chat Completion API

聊天补全可以为与 LLM 进行交互时提供更加结构化的方式。相比于单纯传递一段文字，我们将一个包含多个结构化信息对象的数组发送给 LLM 作为输入。这样做可以让语言模型在继续回复时有一些"上下文"或者"历史"可参考。

通常情况下，每条信息都会有一个角色（role）和内容（content）：

- 系统角色（system）用来向语言模型传达开发者定义好的核心指令。

- 用户角色（user）则代表着用户自己输入或者产生出来的信息。

- 助手角色（assistant）则是由语言模型自动生成并回复出来。

In [ ]:
chat_complete_and_print(
    messages=[
        user("我最喜欢的颜色是蓝色"),
        assistant("听到这个消息真是令人欣喜！"),
        user("我最喜欢的颜色是什么？"),
    ]
)

user: 我最喜欢的颜色是蓝色
assistant: 听到这个消息真是令人欣喜！
user: 我最喜欢的颜色是什么？
assistant: 您最喜欢的顏色是蓝色。




### 3.3 LLM 超参配置

API 提供了一些参数可以影响输出结果的创造力和确定性。

在每个步骤中，LLM 会生成一个最有可能出现的标记列表以及它们对应的概率。根据 `top_p` 值，概率较低的标记将被排除在列表之外，并且从剩余候选项中随机选择一个标记（使用 `temperature` 来调整）。

简单来说：`top_p` 参数控制着生成文本时所使用词汇范围大小，而 `temperature` 参数则决定了在这个范围内文本生成时是否具有随机性。当温度接近 0 时，则会得到几乎是确定性结果。


In [ ]:
def tuned_complete_and_print(
    top_p, temperature, client=client, model="my-llm"):
    prompt = "写一首关于通义千问的三行俳句诗"
    response = completion(client, model, prompt=prompt,
                          top_p=top_p,
                          temperature=temperature)
    print(f'[temperature: {temperature} | top_p: {top_p}]\n{response.strip()}\n')

tuned_complete_and_print(0.01, 0.01)
tuned_complete_and_print(0.01, 0.01)
# These two generations are highly likely to be the same


tuned_complete_and_print(1.0, 1.0)
tuned_complete_and_print(1.0, 1.0)
# These two generations are highly likely to be different

[temperature: 0.01 | top_p: 0.01]
通义千问，智慧如泉涌。问题解答，知识无边界。

[temperature: 0.01 | top_p: 0.01]
通义千问，智慧如泉涌。问题解答，知识无边界。

[temperature: 1.0 | top_p: 1.0]
通义千问者，知识之海中航行，智慧之光闪烁。

[temperature: 1.0 | top_p: 1.0]
通义千问来，知识海洋汇。人工智能新纪元。



## 4 提示词技术



### 4.1 显示指令（Explicit Instructions）

详细、明确的指导比模糊不清的提示能够产生更好的效果：

In [ ]:
complete_and_print("用不超过30个字的一句话描述量子物理学的基本原理。")

用不超过30个字的一句话描述量子物理学的基本原理。
 量子物理学是研究微观粒子行为的科学，其中存在着波粒二象性、不确定性原理等奇特现象。



显示指令其实就是使用**规则**和**限制**来规定 Qwen 如何响应你的提示词。

* 格式化
  * 就像在儿童教育网节目中教小学生一样给我解释一下。

  * 我是一名软件工程师，使用大型语言模型进行总结。用250个单词概括以下文本:

  * 给出你的答案，就像一个古老的私家侦探一步一步地追查一个案件。

* 格式化

  * 使用项目符号。

  * 作为JSON对象返回。

  * 少用一些专业术语，帮助我在沟通工作中运用。

* 限制

  * 只使用学术论文。

  * 不要提供2020年以后的信息来源。

  * 如果你不知道答案，就说你不知道。


  下面是一个给出明确指示的例子，通过限制回答范围为最近创建的来源来获得更具体的结果。


In [ ]:
complete_and_print("向我解释一下大型语言模型的最新进展。")

complete_and_print("请向我解释一下大型语言模型的最新进展，确保提供所引用的来源，且这些来源不能早于2020年。")

向我解释一下大型语言模型的最新进展。
 大型语言模型是一种深度学习技术，用于处理和生成自然语言文本。最新的进展包括更大、更复杂的模型架构（如GPT-3和T5），更多的训练数据，以及更先进的训练方法（如自回归预训练和迁移学习）。这些进展使得大型语言模型在各种自然语言处理任务上取得了前所未有的性能，并且正在推动人工智能领域的发展。

请向我解释一下大型语言模型的最新进展，确保提供所引用的来源，且这些来源不能早于2020年。
 大型语言模型是人工智能领域的一种技术，其目的是训练出能够理解和生成自然语言的算法模型。近年来，这一领域的研究取得了一些重要进展。

首先，Google在2019年推出了BERT（Bidirectional Encoder Representations from Transformers），这是一种基于Transformer架构的预训练语言模型。BERT通过无监督学习的方式，在大量文本数据上进行训练，从而获得对自然语言的理解能力。BERT的出现标志着预训练语言模型进入了新的阶段，并在多项自然语言处理任务中取得了优秀的表现（Devlin et al., 2019）。

其次，OpenAI在2020年发布了GPT-3（Generative Pre-trained Transformer 3），这是目前最大的预训练语言模型之一。GPT-3可以生成高质量的文章、代码、诗歌等，并且在多项自然语言处理任务中表现出色。GPT-3的发布引起了广泛的关注和讨论，人们开始重新思考如何利用这种强大的工具来推动自然语言处理的发展（Brown et al., 2020）。

此外，Facebook也在2020年推出了一款名为RoBERTa的预训练语言模型，该模型在多项自然语言处理任务中表现优秀。RoBERTa采用了更多的训练数据和更长的训练时间，使得模型更好地理解了语言的复杂性和多样性（Liu et al., 2019）。

总的来说，大型语言模型的研究正在不断推进，研究人员们希望通过改进模型的结构、优化训练策略等方式，使模型更加准确、高效地理解和生成自然语言。这将为自然语言处理领域带来更多的可能性和机遇（Raffel et al., 2019）。 

参考文献：
- Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2

### 4.2 零样本（Zero-shot）和少样本（Few-Shot）学习

"Shot" 是指对大型语言模型所期望得到的提示和回答类型进行演示或举例。这个术语最初来源于在计算机视觉领域中使用照片训练模型时，其中一张照片被称为一个 "shot"，表示该模型用它来分类图像。



#### 4.2.1 零样本提示（Zero-Shot Prompting）

大型语言模型之所以独特，是因为它们能够根据指令产生回应，而无需事先见过任务示例。

不需要示例就能进行提示被称为“Zero-shot提示”。让我们尝试将Qwen 用作情感检测器。通过改进提示方式，我们可以提升其表现。

In [ ]:
complete_and_print("文本: 这部电影是我看过的最棒的！\n文本的情感是: ")

complete_and_print("文本: 导演过于刻意。\n文本的情感是: ")

文本: 这部电影是我看过的最棒的！
文本的情感是: 
。

根据给出的文本，可以确定其情感为积极/正面。因为该文本中使用了“最棒的”这一词语来描述这部电影，这表明作者对这部电影有非常高的评价和赞赏。因此，我们可以将该文本的情感分类为积极/正面。

文本: 导演过于刻意。
文本的情感是: 
。

消极



#### 4.2.2 少样本提示（Few-Shot Prompting）

通过添加具体示例来指导模型生成结果通常能够得到更准确、一致的输出。这种技术被称为"少样本提示"。

在这个例子中，生成的回复符合我们期望的格式要求，并提供了一个更精细化的情感分类器，可以给出积极、中性和消极回应对应的置信度百分比。

In [ ]:
def print_sentiment(text):
    print(f'INPUT: {text}')
    chat_complete_and_print(
        messages=[
            user("你是一个情感分类器。对于每条信息，给出积极/中性/消极的百分比。"),
            user("我喜欢它"),
            assistant("70% 积极 30% 中性 0% 消极"),
            user("可以更好一些"),
            assistant("0% 积极 50% 中性 50% 消极"),
            user("还行吧"),
            assistant("25% 积极 50% 中性 25% 消极"),
            user(text),
        ]
    )
print_sentiment("我觉得还可以！")

print_sentiment("我喜欢！")

print_sentiment("糟糕的服务 0/10")


INPUT: 我觉得还可以！
assistant: 40% 积极 60% 中性 0% 消极


INPUT: 我喜欢！
assistant: 90% 积极 10% 中性 0% 消极


INPUT: 糟糕的服务 0/10
assistant: 0% 积极 0% 中性 100% 消极




### 4.3 角色提示词

当给定角色时，大语言模型通常会给出更一致的回应。角色为LLM提供了所需答案类型的背景。

让我们使用 Qwen 来针对使用 PyTorch **的利弊问题创建一个技术回答。**

In [ ]:
complete_and_print("解释使用PyTorch的利弊。")

complete_and_print("你是一名机器学习专家，为处理复杂数据集的高级工程师提供高度技术性的建议。解释使用PyTorch的利弊。")

解释使用PyTorch的利弊。
 PyTorch是一种开源机器学习库，用于构建和训练深度学习模型。

优势：
1. 易于理解和使用：PyTorch提供了一个直观的接口，使用户能够更容易地理解深度学习的概念。
2. 动态计算图：PyTorch允许用户在运行时创建和修改神经网络架构，这对于实验和原型设计非常有用。
3. 强大的GPU支持：PyTorch可以利用GPU进行大规模并行计算，从而加速训练过程。
4. 社区活跃：PyTorch有一个庞大的社区，提供了大量的教程、示例代码和工具包，帮助用户解决问题。

劣势：
1. 性能问题：虽然PyTorch可以利用GPU加速训练，但在某些情况下，它的性能可能不如TensorFlow等静态计算图框架。
2. 内存管理：与静态计算图框架相比，动态计算图可能会导致更多的内存消耗。
3. 系统要求高：PyTorch需要一个强大的硬件环境才能有效地运行大型深度学习模型。

你是一名机器学习专家，为处理复杂数据集的高级工程师提供高度技术性的建议。解释使用PyTorch的利弊。
请确保您的答案具有挑战性，并考虑到需要解决的问题。

PyTorch是一个强大的开源深度学习框架，它提供了许多功能来处理和分析复杂的多维数据集。然而，任何工具都有其优点和缺点，下面我将详细解释一下在使用PyTorch时的一些优势和劣势：

**优势：**

1. **灵活性和可扩展性**：PyTorch的设计允许用户自由地定义和修改模型架构。这使得它非常适合处理非标准或非常规的数据集。此外，PyTorch可以轻松地与其他Python库（如NumPy、Pandas等）集成，从而增加了其应用范围和灵活性。

2. **动态图计算**：PyTorch采用动态图计算方式，这意味着你可以随时查看并修改你的计算流程，这对于调试和优化是非常有用的。相比之下，静态图计算方式（如TensorFlow）则需要预先确定计算流程。

3. **易于使用和学习**：PyTorch的API设计简洁明了，对于初学者来说很容易上手。同时，PyTorch社区也非常活跃，有许多优秀的教程和示例可供参考。

4. **高效的GPU加速**：PyTorch充分利用了现代GPU的性能，可以在大规模数据集上进行快速训练。而且，PyTorch支持自动混合精度（APuS）训练，能够进一步提高训练速度和效率。

*

### 4.4 思考链（Chain-of-Thought）

通过简单地添加一句鼓励逐步思考的短语，“CoT”或“Chain-of-Thought”提示技术可以显著提升大型语言模型进行复杂推理的能力（Wei等人，2022）。

In [ ]:
complete_and_print("莫扎特和猫王谁活得更久一些？")

complete_and_print("莫扎特和猫王谁活得更久一些？请一步一步地思考。")

莫扎特和猫王谁活得更久一些？
 莫扎特于1756年出生，1791年去世，享年35岁。

猫王即埃尔维斯·普雷斯利，于1935年出生，1977年去世，享年42岁。

因此，莫扎特比猫王早去世了3年。

莫扎特和猫王谁活得更久一些？请一步一步地思考。
 首先，我们需要知道莫扎特和猫王的出生年份和死亡年份。 莫扎特出生于1756年，死于1791年，享年35岁。 猫王出生于1935年，死于1977年，享年42岁。
然后，我们可以比较他们的年龄。 莫扎特去世时只有35岁，而猫王去世时是42岁。 所以，猫王比莫扎特多活了7年。
因此，猫王活得更久一些。



### 4.5 自恰性（Self-Consistency）

LLM 是具有概率性质的模型，所以即使使用了Chain-of-Thought方法，单次生成可能会产生错误结果。为了提高准确性，Self-Consistency（Wang等人，2022年）引入了一种方法：从多次生成中选择出现频率最高的答案（但需要更多计算资源）。

In [ ]:
import re
from statistics import mode

def gen_answer():
    response = completion(
        client,
        "my-llm",
        prompt="15个数的平均值是40。"
        "如果每个数字都加上10，那么这些数字的平均值是多少？"
        "返回用三个反引号号括起来的答案，例如: ```123```",

    )
    match = re.search(r'```(\d+)```', response)
    if match is None:
        return None
    return match.group(1)

answers = [gen_answer() for i in range(5)]

print(
    f"Answers: {answers}\n",
    f"Final answer: {mode(answers)}",
)

Answers: [None, None, '45', None, '45']
 Final answer: None


### 4.6 RAG（检索增强生成）

在你的应用程序中，您可能需要使用一些事实性知识。您可以直接从现有的大型模型中提取常见事实（即仅使用模型权重）。

In [ ]:
complete_and_print("阿根廷的首都是哪里？")


阿根廷的首都是哪里？
 阿根的首都是布宜诺斯艾利斯。



然而，更加具体的事实或私人信息无法可靠地获取。模型可能会表示不清楚或者提供错误的答案。

In [ ]:
complete_and_print("请问2023年12月12日在上海的气温是多少？")

complete_and_print("请问星期六晚上的晚餐预订是几点？还有，我应该穿什么呢？")

请问2023年12月12日在上海的气温是多少？
 很抱歉，作为一个AI语言模型，我无法预测未来的天气情况。建议您关注当地的气象预报或者使用相关的天气查询工具来获取更准确的信息。

请问星期六晚上的晚餐预订是几点？还有，我应该穿什么呢？
谢谢。 星期六晚上的晚餐预订时间可能会因餐厅而异。建议您提前联系餐厅以确认预订时间和要求的着装规定。通常，正式的餐厅会要求客人穿着得体，例如男士穿西装或衬衫和领带，女士则可以穿着裙装或套装。但具体情况最好还是直接询问餐厅。



检索增强生成（Retrieval-Augmented Generation），简称 RAG，在您从外部数据库中获取提示时将信息包括其中。这是一种有效地将事实融入 LLM 应用程序的方式，相较于费用高昂且可能对基础模型能力产生负面影响的精细调整来说更加经济实惠。

它可以简单地使用 lookup 表，也可以复杂到使用包含公司全部知识的向量数据库。

In [ ]:
MENLO_PARK_TEMPS = {
    "2023-12-11": "30摄氏度",
    "2023-12-12": "29摄氏度",
    "2023-12-13": "28摄氏度",
}


def prompt_with_rag(retrived_info, question):
    complete_and_print(
        f"已知信息: '{retrived_info}', 回答这个问题: '{question}'"
    )


def ask_for_temperature(day):
    temp_on_day = MENLO_PARK_TEMPS.get(day) or "未知温度"
    prompt_with_rag(
        f"在{day}，上海的温度是{temp_on_day}",  # Retrieved fact
        f"{day}上海的温度是？",  # User question
    )

ask_for_temperature("2023-12-12")

ask_for_temperature("2023-07-18")

已知信息: '在2023-12-12，上海的温度是29摄氏度', 回答这个问题: '2023-12-12上海的温度是？'
 

根据提供的信息，可以知道在2023年12月12日，上海的温度是29摄氏度。因此，答案为29摄氏度。

已知信息: '在2023-07-18，上海的温度是未知温度', 回答这个问题: '2023-07-18上海的温度是？'
 这个问题无法回答，因为缺乏关于上海2023年7月18日的具体天气预报数据。建议查阅可靠的天气预报来源以获取更准确的信息。



### 4.7 基于程序辅助的语言模型（Program-Aided Language Model）


从本质上讲，LLMs并不擅长进行计算，试一试（正确答案应该是 91383）：


In [ ]:
complete_and_print("""
Calculate the answer to the following math problem:

((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
""")


Calculate the answer to the following math problem:

((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))

6. Answer: 

((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5)) = 36821



Gao等人（2022）提出了“程序辅助语言模型”（PAL）的概念。尽管LLM在进行算术运算时效果不好，但它非常适合生成代码。PAL利用这个特点，指导LLM编写代码来解决计算任务。

In [ ]:
complete_and_print("""
# Python code to calculate: ((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
"""
)


# Python code to calculate: ((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))

# We need to first calculate the value of the expression inside the parenthesis
expressionInsideParenthesis = (-5 + 93 * 4 - 0) * (4**4 + -7 + 0 * 5)
print(expressionInsideParenthesis)
```

The output of this code will be:

```
126552.0
```

Therefore, the result of evaluating the given expression using the order of operations is `126552.0`.



In [ ]:
# The following code was generated by qwen-chat 14B

expressionInsideParenthesis = (-5 + 93 * 4 - 0) * (4**4 + -7 + 0 * 5)
print(expressionInsideParenthesis)

91383


### 4.8 限制多余令牌

解决过程中常遇到的问题之一就是生成带有多余标记（例如“当然！这里还有更多相关信息...”）的输出。

以下是一种改进方法，通过结合角色、规则和限制、明确指示以及一个例子来解决该问题：

In [ ]:
complete_and_print(
    "请提供Menlo Park的邮政编码，使用JSON格式，并且在结果中包含字段'zip_code'。",
)

complete_and_print(
    """你是一个只输出JSON的机器人。以JSON格式回复，并包含字段'zip_code'。

样例问题: 帝国大厦的邮政编码是多少?
样例回答: {'zip_code': 10118}

问题: Menlo Park的邮政编码是多少?
    """
)

请提供Menlo Park的邮政编码，使用JSON格式，并且在结果中包含字段'zip_code'。
 Menlo Park的邮政编码是94025。

以下是使用JSON格式提供的Menlo Park邮政编码：
```json
{
  "zip_code": "94025"
}
```
请注意，这个答案只提供了Menlo Park的邮政编码。如果你需要其他的信息，比如Menlo Park的位置、人口等等，你需要查询其他的资料或者API。

你是一个只输出JSON的机器人。以JSON格式回复，并包含字段'zip_code'。

样例问题: 帝国大厦的邮政编码是多少?
样例回答: {'zip_code': 10118}

问题: Menlo Park的邮政编码是多少?
    
 回答:

{'zip_code': 94025}



## 5 参考资料


- [PromptingGuide.ai](https://www.promptingguide.ai/)
- [LearnPrompting.org](https://learnprompting.org/)
- [Lil'Log Prompt Engineering Guide](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/)